Phase 2 : Analyse de données

In [7]:
1+1

2

In [8]:
from filmsapisdk import MovieClient, MovieConfig

# Se familiariser avec l'API

In [9]:
# Configuration avec l'url de l'API (Render)
config = MovieConfig(movie_base_url="https://datatech.onrender.com")
client = MovieClient(config=config)

MOVIE_API_BASE_URL in MovieConfig init: https://datatech.onrender.com


In [10]:
health = client.health_check()
print("Health check :", health)

Health check : {'Message': 'API MovieLens est opérationnelle!'}


In [11]:
movies_df = client.list_movies(limit=5, output_format="pandas")
print("Liste des films (DataFrame) :")
print(movies_df.head())

Liste des films (DataFrame) :
   movieId  ...                                       genres
0        1  ...  Adventure|Animation|Children|Comedy|Fantasy
1        2  ...                   Adventure|Children|Fantasy
2        3  ...                               Comedy|Romance
3        4  ...                         Comedy|Drama|Romance
4        5  ...                                       Comedy

[5 rows x 3 columns]


In [12]:
ratings_df = client.list_ratings(limit=5, output_format="pandas")
print("Liste des notes (DataFrame) :")
print(ratings_df.head())

Liste des notes (DataFrame) :
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [13]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     5 non-null      int64  
 1   movieId    5 non-null      int64  
 2   rating     5 non-null      float64
 3   timestamp  5 non-null      int64  
dtypes: float64(1), int64(3)
memory usage: 292.0 bytes


In [14]:
analytics = client.get_analytics()
print("Analytics :", analytics)


Analytics : movie_count=9742 rating_count=100836 tag_count=3683 link_count=9742


In [17]:
total_ratings = analytics.rating_count
print("Total des notes :", total_ratings)

Total des notes : 100836


In [23]:
### vu que l'API limit le nombre de requetes afin d'eviter de surchager le server, on va faire un travail par batch de 1000 enregistrements
# du traitement par lot, du batch processing
import time
import pandas as pd


batch_size = 1000
all_ratings = []
for skip in range(0, total_ratings, batch_size):
    print(f"telechargement des lignes {skip} à {skip + batch_size, total_ratings}...")
    batch_df = client.list_ratings(limit=batch_size, skip=skip, output_format="pandas")
    all_ratings.append(batch_df)
    time.sleep(0.5)  # Pause de 0.5 seconde entre les requêtes pour respecter les limites de l'API

    # Combiner tous les DataFrames en un seul
ratings_full_df = pd.concat(all_ratings, ignore_index=True)

print("Total des notes téléchargées :", len(ratings_full_df))

telechargement des lignes 0 à (1000, 100836)...
telechargement des lignes 1000 à (2000, 100836)...
telechargement des lignes 2000 à (3000, 100836)...
telechargement des lignes 3000 à (4000, 100836)...
telechargement des lignes 4000 à (5000, 100836)...
telechargement des lignes 5000 à (6000, 100836)...
telechargement des lignes 6000 à (7000, 100836)...
telechargement des lignes 7000 à (8000, 100836)...
telechargement des lignes 8000 à (9000, 100836)...
telechargement des lignes 9000 à (10000, 100836)...
telechargement des lignes 10000 à (11000, 100836)...
telechargement des lignes 11000 à (12000, 100836)...
telechargement des lignes 12000 à (13000, 100836)...
telechargement des lignes 13000 à (14000, 100836)...
telechargement des lignes 14000 à (15000, 100836)...
telechargement des lignes 15000 à (16000, 100836)...
telechargement des lignes 16000 à (17000, 100836)...
telechargement des lignes 17000 à (18000, 100836)...
telechargement des lignes 18000 à (19000, 100836)...
telechargement 

In [24]:
# Agrégation : nombre d'évaluations par utilisateur
ratings_per_user = ratings_full_df['userId'].value_counts().rename_axis('userId').reset_index(name='rating_count')

#affichage du resultat
ratings_per_user


,userId,rating_count
0,414,2698
1,599,2478
2,474,2108
3,448,1864
4,274,1346
...,...,...
605,442,20
606,278,20
607,147,20
608,320,20


In [25]:
# Agreger ratings_count par userId , chunk par chunk
total_ratings = client.get_analytics().rating_count
batch_size = 1000

from collections import defaultdict
user_rating_counts = defaultdict(int)

# Parcours des chunks
for skip in range(0, total_ratings, batch_size):
    print(f"Processing rows {skip} to {min(skip + batch_size, total_ratings)}...")
    batch_df = client.list_ratings(limit=batch_size, skip=skip, output_format="pandas")
    
    # Compter les évaluations par utilisateur dans le chunk actuel
    batch_counts = batch_df['userId'].value_counts()
    
    # Mettre à jour les comptes totaux
    for user_id, count in batch_counts.items():
        user_rating_counts[user_id] += count
    time.sleep(0.5)  # Pause de 0.5 seconde entre les requêtes pour respecter les limites de l'API

# Convertir le résultat en DataFrame
user_rating_counts_df = pd.DataFrame(list(user_rating_counts.items()), columns=['userId', 'rating_count'])

# 6 Affichage du resultat
user_rating_counts_df

Processing rows 0 to 1000...
Processing rows 1000 to 2000...
Processing rows 2000 to 3000...
Processing rows 3000 to 4000...
Processing rows 4000 to 5000...
Processing rows 5000 to 6000...
Processing rows 6000 to 7000...
Processing rows 7000 to 8000...
Processing rows 8000 to 9000...
Processing rows 9000 to 10000...
Processing rows 10000 to 11000...
Processing rows 11000 to 12000...
Processing rows 12000 to 13000...
Processing rows 13000 to 14000...
Processing rows 14000 to 15000...
Processing rows 15000 to 16000...
Processing rows 16000 to 17000...
Processing rows 17000 to 18000...
Processing rows 18000 to 19000...
Processing rows 19000 to 20000...
Processing rows 20000 to 21000...
Processing rows 21000 to 22000...
Processing rows 22000 to 23000...
Processing rows 23000 to 24000...
Processing rows 24000 to 25000...
Processing rows 25000 to 26000...
Processing rows 26000 to 27000...
Processing rows 27000 to 28000...
Processing rows 28000 to 29000...
Processing rows 29000 to 30000...
Pr

,userId,rating_count
0,6,314
1,1,232
2,4,216
3,7,152
4,5,44
...,...,...
605,604,100
606,608,831
607,607,187
608,610,1302


In [ ]:
# Par ordre decroissant
user_rating_counts_df = user_rating_counts_df.sort_values(by='rating_count', ascending=False)
user_rating_counts_df

,userId,rating_count
410,414,2698
597,599,2478
471,474,2108
438,448,1864
268,274,1346
...,...,...
447,442,20
199,194,20
206,207,20
321,320,20


# Question business : quels genres de films les utilisateurs taguent le plus positivement ?

Hypothese : la note est positive si elle est superieure ou egale 4.0(subjective)

quels sont les tags les plus frequents associés à ces genres ?


- ratings : filtrer les evaluations élévées (>=4.0 )
- tags : pour voir quels tags sont utilisés sur les films les mieux notés
- films

In [28]:
# Étape 1 : Recuperer les évaluations élévées (rating >= 4) par lots

chunk_size = 500
skip = 0

all_high_ratings = []

while True:
    chunk = client.list_ratings(
        limit=chunk_size,
        skip=skip,
        min_rating=4.0,
        output_format="pandas")
    if chunk.empty:
        break
    all_high_ratings.append(chunk[chunk['rating'] >= 4])
    skip += chunk_size
    time.sleep(0.5)  # Pause de 0.5 seconde entre les requêtes pour respecter les limites de l'API
# Combiner tous les chunks en un seul DataFrame
high_ratings_df = pd.concat(all_high_ratings, ignore_index=True)
print("Total des évaluations élevées (rating >= 4) :", len(high_ratings_df))
high_ratings_df


Total des évaluations élevées (rating >= 4) : 48580


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
48575,610,166528,4.0,1493879365
48576,610,166534,4.0,1493848402
48577,610,168248,5.0,1493850091
48578,610,168250,5.0,1494273047


In [29]:
# Étapes 2 : Identifier les couples (userId, movieId) uniques (pour eviter les doublons)
unique_user_movie_pairs = high_ratings_df[['userId', 'movieId']].drop_duplicates()
print("Total des couples (userId, movieId) uniques avec des évaluations élevées :", len(unique_user_movie_pairs))
unique_user_movie_pairs

Total des couples (userId, movieId) uniques avec des évaluations élevées : 48580


,userId,movieId
0,1,1
1,1,3
2,1,6
3,1,47
4,1,50
...,...,...
48575,610,166528
48576,610,166534
48577,610,168248
48578,610,168250


In [35]:
# Étape 3 : Récupérer les tags correspondants à ces couples
alls_tags = []
skip = 0
chunk_size = 1000
while True:
    tags_chunks = client.list_tags(limit=chunk_size, skip=skip, output_format="pandas")
    if tags_chunks.empty:
        break
    alls_tags.append(tags_chunks)
    skip += chunk_size
    time.sleep(0.5)  # Pause de 0.5 seconde entre les requêtes pour respecter les limites de l'API

alls_tags_df = pd.concat(alls_tags, ignore_index=True)

alls_tags_df

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,89774,Boxing story,1445715207
2,2,106782,drugs,1445715054
3,7,48516,way too long,1169687325
4,18,431,Al Pacino,1462138765
...,...,...,...,...
1773,606,6107,World War II,1178473747
1774,606,7382,for katie,1171234019
1775,606,7936,austere,1173392334
1776,610,3265,gun fu,1493843984


In [36]:
# Merger les tags avec les couples (userId, movieId) uniques
merged_tags = pd.merge(unique_user_movie_pairs, alls_tags_df, on=['userId', 'movieId'])
print(merged_tags.shape)
merged_tags

(1020, 4)


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,89774,Boxing story,1445715207
2,2,106782,drugs,1445715054
3,18,431,Al Pacino,1462138765
4,18,1221,Al Pacino,1461699306
...,...,...,...,...
1015,606,1357,music,1176765393
1016,606,6107,World War II,1178473747
1017,606,7382,for katie,1171234019
1018,610,3265,gun fu,1493843984


In [37]:
# Étape 4 : Récupérer les genres associés au movieId

def get_movie_genres(movie_id):
    try:
        movie = client.get_movie(movie_id)
        return movie.genres
    except Exception as e:
        print(f"Erreur lors de la récupération des genres pour movieId {movie_id} : {e}")
        return ""

# Appliquer la fonction pour chaque movieId unique dans merged_tags
unique_movie_ids = merged_tags['movieId'].unique()

movie_genres = {
    movie_id: get_movie_genres(movie_id)
    for movie_id in unique_movie_ids
}

# Ajouter une colonne 'genres' à merged_tags
merged_tags['genres'] = merged_tags['movieId'].map(movie_genres)
print(merged_tags.shape)
merged_tags

(1020, 5)


,userId,movieId,tag,timestamp,genres
0,2,60756,funny,1445714994,Comedy
1,2,89774,Boxing story,1445715207,Drama
2,2,106782,drugs,1445715054,Comedy|Crime|Drama
3,18,431,Al Pacino,1462138765,Crime|Drama
4,18,1221,Al Pacino,1461699306,Crime|Drama
...,...,...,...,...,...
1015,606,1357,music,1176765393,Drama|Romance
1016,606,6107,World War II,1178473747,Drama|War
1017,606,7382,for katie,1171234019,Drama|Mystery|Thriller
1018,610,3265,gun fu,1493843984,Action|Crime|Drama|Thriller


In [38]:
# Étape 5 : Agrégation finale : genre <-> tags <-> count

# On va exploser les genres s'ils sont separés par des '|'
merged_tags['genres'] = merged_tags['genres'].str.split('|')
exploded_tags = merged_tags.explode('genres')

exploded_tags

,userId,movieId,tag,timestamp,genres
0,2,60756,funny,1445714994,Comedy
1,2,89774,Boxing story,1445715207,Drama
2,2,106782,drugs,1445715054,Comedy
2,2,106782,drugs,1445715054,Crime
2,2,106782,drugs,1445715054,Drama
...,...,...,...,...,...
1018,610,3265,gun fu,1493843984,Drama
1018,610,3265,gun fu,1493843984,Thriller
1019,610,168248,Heroic Bloodshed,1493844270,Action
1019,610,168248,Heroic Bloodshed,1493844270,Crime


In [39]:
# Compter les combinaisons genre-tag
genre_tag_summary = (exploded_tags
.groupby(['genres', 'tag'])
.size()
.reset_index(name='count')
.sort_values(by='count', ascending=False)
)
genre_tag_summary

,genres,tag,count
754,Drama,In Netflix queue,19
285,Children,Disney,12
853,Drama,atmospheric,12
247,Animation,Disney,11
417,Comedy,black comedy,9
...,...,...,...
1775,War,journalism,1
1776,War,made me cry,1
1777,War,military,1
1778,War,missionary,1
